# Raport z implementacji modeli uczenia maszynowego

## 1. Wstęp
**Cel projektu**:  
Klasyfikacja stanów otyłości (7 klas) na podstawie nawyków żywieniowych i parametrów fizycznych.  
**Zbiór danych**:  
obesity_data.csv (2111 próbek, 17 cech)  

## 2. Szczegółowa analiza preprocessingu

### 2.1 Struktura transformacji
Pipeline przetwarzania wykorzystuje `ColumnTransformer` do równoległego przetwarzania różnych typów cech:


### 2.2 Rodzaje transformacji
1. **Cechy binarne** (np. płeć):
   - Kodowanie 0/1 przez `OrdinalEncoder`
   - Przykład: "Male" → 0, "Female" → 1

3. **Cechy kategoryczne** (sposób przemieszczania się):
- Pełne one-hot encoding przez `OneHotEncoder`

### 2.3 Skalowanie
- `RobustScaler` stosowany globalnie po transformacji kategorycznej
- Odporny na outliery poprzez użycie mediany

### 2.4 Wyniki modeli scikit-learn
| Model                  | Dokładność |
|------------------------|------------|
| DecisionTree           | 60.9%      |
| LogisticRegression     | 53.9%      |
| SVC                    | 62.4%      |

## 3. Implementacje w numpy 
### 3.1 Regresja zamkniętą formułą

**Ograniczenia**:  
- Brak stabilności numerycznej dla `X.T @ X` przy dużych wymiarach
- Podatność na outliery
- Nieefektywne n^3
- Nie radzi sobie dobrze z danymi, które mają nieliniową strukturę

### 3.2 Regresja logistyczna z GD

**Ograniczenia**:  
- Kluczowe hiperparametry, jak learning rate czy liczba epok, trzeba dobrać ręcznie lub metodami optymalizacji.
- Gradient descent minimalizuje błąd ogólny, więc przy dużej dysproporcji klas może ignorować mniejszościową klasę.

### 3.3 Porównanie wydajności
| Metoda                 | Dokładność |
|------------------------|------------|
| Własna regresja        | 14.4%      |
| Scikit-learn           | 53.9%      |

**Analiza**:  
Różnica wynika z braku optymalizacji w implementacji własnej (brak regularyzacji, prosty GD)

## 4. Implementacja w PyTorch 

### 4.1 Wyniki GPU vs CPU
| Metryka                | CPU      | GPU      |
|------------------------|----------|----------|
| Czas                   | 9.73s    | 13.1s    |

**Analiza**:
Powody dlaczego czas trenowania na cpu jest mniejszy niż na gpu
- Mały rozmiar danych – GPU zyskuje przewagę przy dużych macierzach, dla małych zbiorów dane mogą się szybciej przetwarzać na CPU ze względu na mniejsze koszty narzutu.
- Overhead związany z inicjalizacją GPU – uruchomienie kerneli GPU i zarządzanie nimi może zająć więcej czasu niż bezpośrednie wykonanie kodu na CPU.
- Niewielka liczba operacji matematycznych – regresja logistyczna to stosunkowo prosty model, który nie wykorzystuje w pełni równoległych możliwości GPU.
- Koszt przenoszenia danych do/z GPU – kopiowanie danych między pamięcią RAM (CPU) a pamięcią GPU może zająć więcej czasu niż samo obliczenie na CPU.


